In [ ]:
# Packages 
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from sklearn.metrics import f1_score

## Deep SAD(Deep semi-supervised anomaly detection)
본 노트북의 코드는 논문 *"Deep Semi-Supervised Anomlay Detection(Lukas Ruff et al. ICLR 2020)"* 의 이론 및 알고리즘을 바탕으로 작성되었으며, pytorch로 작성된 해당 논문의 코드(https://github.com/lukasruff/Deep-SAD-PyTorch/blob/master/src/DeepSAD.py)를 참고하여 tensorflow로 작성하였다.

### DeepSVDD
DeepSAD 알고리즘은 딥러닝을 이용한 anomaly detection 알고리즘인 Deep SVDD(Ruff et al., 2018)을 Semi-supervised learning, 즉 본 문제와 같이 labeled sample이 추가로 주어진 상황으로 확장시킨 알고리즘이다. 주어진 $n$개의 unlabeled training samples $x_1,\ldots,x_n\in\mathcal X$에서 Deep SVDD의 목적함수는 다음과 같다.
$$
\min_W{1\over n}\sum_{i=1}^n\Vert\phi(x_i;W)-c\Vert^2 + {\lambda\over 2}\sum_{l=1}^L\Vert W^l\Vert_F^2,\;\;\lambda>0
$$
즉, hypersphere의 중심점인 상수벡터 c를 중심으로 mean squared distance를 최소화하는 알고리즘이므로, 이는 곧 데이터들의 공통된 특성을 추출하는 과정으로 볼 수 있다. 최적화 과정은 SGD를 이용한 backpropagation으로 이루어지며 trained model에 대한 anomaly score는
$$
s(x) = \Vert \phi(x;W)-c\Vert
$$
로 주어진다.

### DeepSAD
반면, DeepSAD의 경우 semi-supervised learning 문제이므로 labeled data를 최적화 식에 반영하여 다음과 같은 목적함수를 취한다.
$$
\min_W{1\over n+m}\sum_{i=1}^n\Vert\phi(x_i;W)-c\Vert^2 +{\eta\over n+m}\sum_{j=1}^m(\Vert\phi(\tilde x_j;W)-c\Vert^2)^{\tilde y_j} +{\lambda\over 2}\sum_{l=1}^L\Vert W^l\Vert_F^2,\;\
$$

여기서 두번째 항의 샘플
$$
(\tilde x_1,\tilde y_1),\ldots,(\tilde x_m, \tilde y_m)
$$
이 labeled data에 해당하며, 상수 $\eta$는 이 labeled data의 norm을 얼마나 반영할 것인지 정하는 상수이다. 즉, $\eta>1$이면 unlabeled data보다 labeled data를 더 많이 반영하겠다는 설정이 된다.

In [ ]:
# Deep SAD Algorithm

